In [1]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import time
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import os

In [2]:
ct = datetime.datetime.now()
print("current time:-", ct)

current time:- 2021-06-06 12:07:31.432137


In [3]:
#Check to see if the element exists
def check_exists_by_xpath(xpath):
    try:
        driver.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True

In [4]:
#Static Xpaths
loadMoreXpath = str("//div[@class='search-result-more']/div[@class='search-result-more-txt']")
loadNoMoreXpath = str("//div[@class='search-result-more']/div[@class='search-result-more-txt search-result-no-more']")

In [5]:
#Get list of company names and replace space,& for URL
df_company = pd.read_excel('../../DataSet/CompanyList_Original.xlsx')
all_names = df_company.Name.unique()
print('Total companies : ' + str(len(all_names)))

#Get list of company names and replace space,& for URL
df = pd.read_excel('../../DataSet/Final.xlsx')
scrapped_names = df.company.unique()
print('Companies scraped : ' + str(len(scrapped_names)))

rem_names = [c for c in all_names if c not in scrapped_names]

Total companies : 571
Companies scraped : 248


In [6]:
import random
sample = random.sample(rem_names, 200)
print(sample)

['ENAGAS SA', 'SUNDRUG', 'ALACER GOLD ', 'FIDELITY NATIONAL INFORMATION', 'MARICO ', 'ARENA PHARMACEUTICALS', 'R1 RCM', 'CHINA TOURISM GROUP DUTY FREE', 'PASSAGE BIO', 'LUXSHARE PRECISION INDUSTRY ', 'A-LIVING SERVICES  ', 'CHINA SHIPBUILDING INDUSTRY ', 'COUNTRY GARDEN SERVICES HOLDIN', 'RED ELECTRICA CORP SA', 'MADRIGAL PHARMACEUTICALS', 'GLODON  ', 'ACCENTURE ', 'AMEDISYS', 'GEORGIA HEALTHCARE GROUP ', 'FAST RETAILING', 'KAUFMAN & BROAD SA', 'TIS', 'SEIREN', 'RADIUS HEALTH', 'BEIJING SHUNXIN AGRICULTURE CO', 'SUNAC CHINA HOLDINGS ', 'SANDS CHINA ', 'CHINA CONSTRUCTION BANK ', 'VISCOFAN SA', 'KOJAMO OYJ', 'SAIC MOTOR  ', 'MEDIATEK', 'MEITUAN DIANPING', 'LONGFOR GROUP HOLDINGS ', 'ALLAKOS', 'GEMDALE ', 'SCHAEFFLER INDIA ', 'HYUNDAI MOTOR', 'CHINA MOBILE ', 'MATCH GROUP', 'HARTFORD FINANCIAL SERVICES GR', 'ARES MANAGEMENT ', 'SUPERMAX CORP BHD', 'AU SMALL FINANCE BANK ', 'HINDUSTAN UNILEVER ', 'SOLGOLD ', 'BANK MANDIRI PERSERO TBK PT', 'SANY HEAVY INDUSTRY  ', 'GAKUJO CO ', 'MELIA HOTE

In [7]:
#Get list of company names and replace space,& for URL
name_url = []
for i in sample:
    k = i.replace(" ","+").replace("&","%26").replace("'","%27")
    name_url.append(k)

In [8]:
len(name_url)

200

In [18]:
def extract_to_excel(excel_path, companies_list, start_index, end_index):
    writer = pd.ExcelWriter(excel_path, engine='openpyxl')
    for company in companies_list[start_index:end_index]:
        #Give chrome driver path
        chromedriver = "/usr/local/bin/chromedriver"
        os.environ["webdriver.chrome.driver"] = chromedriver
        driver2 = webdriver.Chrome(chromedriver)
        url = "https://www.reuters.com/search/news?blob="+company+"&sortBy=date&dateRange=pastYear"    
        driver2.get(url)
        
        is_valid = True
        for count in range(100):
            value = check_exists_by_xpath(loadNoMoreXpath)
            if value == True:
                break
            else:
                try:
                    loadMoreButton = driver2.find_element_by_xpath(loadMoreXpath)
                except:
                    is_valid = False
                    break
                loadMoreButton.click()
                time.sleep(8)
                count=count+1
                print("CLICKED :"+str(count))
                
        if not is_valid: continue
        
        titles=[]
        links=[]
        excerpts=[]
        timestamps=[]

        news_titles = driver2.find_elements_by_xpath(".//div[@class='search-result-content']/h3[@class='search-result-title']/a[@href]")
        news_links = driver2.find_elements_by_xpath(".//div[@class='search-result-content']/h3[@class='search-result-title']/a[@href]")
        news_excerpts = driver2.find_elements_by_xpath((".//div[@class='search-result-content']/div[@class='search-result-excerpt']"))
        news_timestamps = driver2.find_elements_by_xpath((".//div[@class='search-result-content']/h5[@class='search-result-timestamp']"))

        for title in news_titles:
            titles.append(title.text)

        for link in news_links:
            links.append(link.get_attribute('href'))

        for excerpt in news_excerpts:
            excerpts.append(excerpt.text)

        for timestamp in news_timestamps:
            timestamps.append(timestamp.text)

        df = pd.DataFrame({
            "title": titles,
            "link": links,
            "excerpt": excerpts,
            "timestamp":timestamps
        })    

        clean_article_body = []
        for i in links:
            response=requests.get(i)
            #print(response.content)
            contents=response.content
            alltext = BeautifulSoup(contents, "lxml")
            #print(alltext)

            article_body=[]
            results = alltext.find_all('p', attrs={'class':'Paragraph-paragraph-2Bgue ArticleBody-para-TD_9x'})
            for d in results:
            #if title is not None:
                article_body.append(d.text.strip())
                cleantext = BeautifulSoup(str(article_body), "lxml").text
                clean_txt = re.sub(r'[?|$|.|!]',r'',cleantext)
            clean_article_body.append(clean_txt)

        df_body = pd.DataFrame({
            "link": links,
            "body": clean_article_body
        })    

        dfinal = df.merge(df_body, on="link", how = 'inner')
        company = company.replace("+"," ").replace("%26","&").replace("%27","'")
        dfinal.to_excel(writer, re.sub('[^A-Za-z0-9 _]*', '', company), index=False)
        driver2.close()
        ct = datetime.datetime.now()
        print(company+"current time:-", ct)

    writer.save()

In [19]:
for counter in range(5):
    excel_path = "../../../Working/Extracted/Scraped_Data_" + str(counter+1) + ".xlsx"
    end_index = 40 * (counter + 1)
    start_index = end_index - 40
    if start_index != 0: start_index += 1
    extract_to_excel(excel_path, name_url, start_index, end_index)

NameError: name 'driver' is not defined

In [44]:
path = "../../../Working/Extracted/Scraped_Data_1.xlsx"
writer = pd.ExcelWriter(path, engine='openpyxl')

for company in name_url[:40]:
    driver = webdriver.Chrome(chromedriver)
    url = "https://www.reuters.com/search/news?blob="+company+"&sortBy=date&dateRange=pastYear"    
    driver.get(url)

    for count in range(100):
        value = check_exists_by_xpath(loadNoMoreXpath)
        if value == True:
            break
        else:
            try:
                loadMoreButton = driver.find_element_by_xpath(loadMoreXpath)
            except:
                print("No results found for " + company)
                break
            loadMoreButton.click()
            time.sleep(8)
            count=count+1
            print("CLICKED :"+str(count))
    
    titles=[]
    links=[]
    excerpts=[]
    timestamps=[]

    news_titles = driver.find_elements_by_xpath(".//div[@class='search-result-content']/h3[@class='search-result-title']/a[@href]")
    news_links = driver.find_elements_by_xpath(".//div[@class='search-result-content']/h3[@class='search-result-title']/a[@href]")
    news_excerpts = driver.find_elements_by_xpath((".//div[@class='search-result-content']/div[@class='search-result-excerpt']"))
    news_timestamps = driver.find_elements_by_xpath((".//div[@class='search-result-content']/h5[@class='search-result-timestamp']"))
    
    for title in news_titles:
        titles.append(title.text)
        
    for link in news_links:
        links.append(link.get_attribute('href'))
        
    for excerpt in news_excerpts:
        excerpts.append(excerpt.text)
        
    for timestamp in news_timestamps:
        timestamps.append(timestamp.text)

    df = pd.DataFrame({
        "title": titles,
        "link": links,
        "excerpt": excerpts,
        "timestamp":timestamps
    })    
    
    clean_article_body = []
    for i in links:
        response=requests.get(i)
        #print(response.content)
        contents=response.content
        alltext = BeautifulSoup(contents, "lxml")
        #print(alltext)
        
        article_body=[]
        results = alltext.find_all('p', attrs={'class':'Paragraph-paragraph-2Bgue ArticleBody-para-TD_9x'})
        for d in results:
        #if title is not None:
            article_body.append(d.text.strip())
            cleantext = BeautifulSoup(str(article_body), "lxml").text
            clean_txt = re.sub(r'[?|$|.|!]',r'',cleantext)
        clean_article_body.append(clean_txt)
    
    df_body = pd.DataFrame({
        "link": links,
        "body": clean_article_body
    })    

    dfinal = df.merge(df_body, on="link", how = 'inner')
    company = company.replace("+"," ").replace("%26","&").replace("%27","'")
    dfinal.to_excel(writer, re.sub('[^A-Za-z0-9 _]*', '', company), index=False)
    #dfinal.to_excel(excel_writer="Extracted/Scraped_Data.xlsx",sheet_name=company)
    driver.close()
    ct = datetime.datetime.now()
    print(company+"current time:-", ct)

writer.save()

CLICKED :1
CLICKED :2
CLICKED :3
CLICKED :4
CLICKED :5
CLICKED :6
CLICKED :7
CLICKED :8
CLICKED :9
CLICKED :10
CLICKED :11
CLICKED :12
CLICKED :13
CLICKED :14
CLICKED :15
CLICKED :16
CLICKED :17
CLICKED :18
SALESFORCE.COMcurrent time:- 2021-06-05 21:08:45.882324
CLICKED :1
CLICKED :2
CLICKED :3
CLICKED :4
CLICKED :5
CLICKED :6
CHINA RAILWAY CONSTRUCTIONcurrent time:- 2021-06-05 21:11:05.146391
CLICKED :1
IAMGOLD current time:- 2021-06-05 21:11:34.000297
CLICKED :1
CLICKED :2
CLICKED :3
ASTELLAS PHARMAcurrent time:- 2021-06-05 21:12:27.682467
No results found for ARGENX+SE
ARGENX SEcurrent time:- 2021-06-05 21:12:35.011178
CONSTELLATION PHARMACEUTICALScurrent time:- 2021-06-05 21:12:49.605341
CLICKED :1
CLICKED :2
CLICKED :3
CLICKED :4
CLICKED :5
CLICKED :6
DBS GROUP HOLDINGScurrent time:- 2021-06-05 21:14:46.534383
No results found for DIVI%27S+LABORATORIES+
DIVI'S LABORATORIES current time:- 2021-06-05 21:14:54.277340
No results found for LANDMARK+OPTOELECTRONICS+
LANDMARK OPTOELECTRO

In [45]:
path = "../../../Working/Extracted/Scraped_Data_2.xlsx"
writer = pd.ExcelWriter(path, engine='openpyxl')

for company in name_url[41:80]:
    driver = webdriver.Chrome(chromedriver)
    url = "https://www.reuters.com/search/news?blob="+company+"&sortBy=date&dateRange=pastYear"    
    driver.get(url)

    for count in range(100):
        value = check_exists_by_xpath(loadNoMoreXpath)
        if value == True:
            break
        else:
            try:
                loadMoreButton = driver.find_element_by_xpath(loadMoreXpath)
            except:
                print("No results found for " + company)
                break
            loadMoreButton.click()
            time.sleep(8)
            count=count+1
            print("CLICKED :"+str(count))
    
    titles=[]
    links=[]
    excerpts=[]
    timestamps=[]

    news_titles = driver.find_elements_by_xpath(".//div[@class='search-result-content']/h3[@class='search-result-title']/a[@href]")
    news_links = driver.find_elements_by_xpath(".//div[@class='search-result-content']/h3[@class='search-result-title']/a[@href]")
    news_excerpts = driver.find_elements_by_xpath((".//div[@class='search-result-content']/div[@class='search-result-excerpt']"))
    news_timestamps = driver.find_elements_by_xpath((".//div[@class='search-result-content']/h5[@class='search-result-timestamp']"))
    
    for title in news_titles:
        titles.append(title.text)
        
    for link in news_links:
        links.append(link.get_attribute('href'))
        
    for excerpt in news_excerpts:
        excerpts.append(excerpt.text)
        
    for timestamp in news_timestamps:
        timestamps.append(timestamp.text)

    df = pd.DataFrame({
        "title": titles,
        "link": links,
        "excerpt": excerpts,
        "timestamp":timestamps
    })    
    
    clean_article_body = []
    for i in links:
        response=requests.get(i)
        #print(response.content)
        contents=response.content
        alltext = BeautifulSoup(contents, "lxml")
        #print(alltext)
        
        article_body=[]
        results = alltext.find_all('p', attrs={'class':'Paragraph-paragraph-2Bgue ArticleBody-para-TD_9x'})
        for d in results:
        #if title is not None:
            article_body.append(d.text.strip())
            cleantext = BeautifulSoup(str(article_body), "lxml").text
            clean_txt = re.sub(r'[?|$|.|!]',r'',cleantext)
        clean_article_body.append(clean_txt)
    
    df_body = pd.DataFrame({
        "link": links,
        "body": clean_article_body
    })    

    dfinal = df.merge(df_body, on="link", how = 'inner')
    company = company.replace("+"," ").replace("%26","&").replace("%27","'")
    dfinal.to_excel(writer, re.sub('[^A-Za-z0-9 _]*', '', company), index=False)
    driver.close()
    ct = datetime.datetime.now()
    print(company+"current time:-", ct)

writer.save()

CLICKED :1
CLICKED :2
CLICKED :3
CLICKED :4
CLICKED :5
CLICKED :6
CLICKED :7
CLICKED :8
CLICKED :9
CLICKED :10
CLICKED :11
CLICKED :12
CLICKED :13
CLICKED :14
CLICKED :15
CLICKED :16
CLICKED :17
CLICKED :18
CLICKED :19
CLICKED :20
CLICKED :21
CLICKED :22
CLICKED :23
CLICKED :24
CLICKED :25
CLICKED :26
CLICKED :27
CLICKED :28
CLICKED :29
CLICKED :30
CLICKED :31
CLICKED :32
CLICKED :33
CLICKED :34
CLICKED :35
CLICKED :36
CLICKED :37
CLICKED :38
CLICKED :39
CLICKED :40
CLICKED :41
CLICKED :42
CLICKED :43
CLICKED :44
CLICKED :45
CLICKED :46
CLICKED :47
CLICKED :48
CLICKED :49
CLICKED :50
CLICKED :51
CLICKED :52
CLICKED :53
CLICKED :54
CLICKED :55
CLICKED :56
CLICKED :57
CLICKED :58
CLICKED :59
CLICKED :60
CLICKED :61
CLICKED :62
CLICKED :63
CLICKED :64
CLICKED :65
CLICKED :66
CLICKED :67
CLICKED :68
CLICKED :69
CLICKED :70
CLICKED :71
CLICKED :72
CLICKED :73
CLICKED :74
CLICKED :75
CLICKED :76
CLICKED :77
CLICKED :78
CLICKED :79
CLICKED :80
CLICKED :81
CLICKED :82
CLICKED :83
CLICKED :84
C

In [46]:
path = "../../../Working/Extracted/Scraped_Data_3.xlsx"
writer = pd.ExcelWriter(path, engine='openpyxl')

for company in name_url[81:120]:
    driver = webdriver.Chrome(chromedriver)
    url = "https://www.reuters.com/search/news?blob="+company+"&sortBy=date&dateRange=pastYear"    
    driver.get(url)

    for count in range(100):
        value = check_exists_by_xpath(loadNoMoreXpath)
        if value == True:
            break
        else:
            try:
                loadMoreButton = driver.find_element_by_xpath(loadMoreXpath)
            except:
                print("No results found for " + company)
                break
            loadMoreButton.click()
            time.sleep(8)
            count=count+1
            print("CLICKED :"+str(count))
    
    titles=[]
    links=[]
    excerpts=[]
    timestamps=[]

    news_titles = driver.find_elements_by_xpath(".//div[@class='search-result-content']/h3[@class='search-result-title']/a[@href]")
    news_links = driver.find_elements_by_xpath(".//div[@class='search-result-content']/h3[@class='search-result-title']/a[@href]")
    news_excerpts = driver.find_elements_by_xpath((".//div[@class='search-result-content']/div[@class='search-result-excerpt']"))
    news_timestamps = driver.find_elements_by_xpath((".//div[@class='search-result-content']/h5[@class='search-result-timestamp']"))
    
    for title in news_titles:
        titles.append(title.text)
        
    for link in news_links:
        links.append(link.get_attribute('href'))
        
    for excerpt in news_excerpts:
        excerpts.append(excerpt.text)
        
    for timestamp in news_timestamps:
        timestamps.append(timestamp.text)

    df = pd.DataFrame({
        "title": titles,
        "link": links,
        "excerpt": excerpts,
        "timestamp":timestamps
    })    
    
    clean_article_body = []
    for i in links:
        response=requests.get(i)
        #print(response.content)
        contents=response.content
        alltext = BeautifulSoup(contents, "lxml")
        #print(alltext)
        
        article_body=[]
        results = alltext.find_all('p', attrs={'class':'Paragraph-paragraph-2Bgue ArticleBody-para-TD_9x'})
        for d in results:
        #if title is not None:
            article_body.append(d.text.strip())
            cleantext = BeautifulSoup(str(article_body), "lxml").text
            clean_txt = re.sub(r'[?|$|.|!]',r'',cleantext)
        clean_article_body.append(clean_txt)
    
    df_body = pd.DataFrame({
        "link": links,
        "body": clean_article_body
    })    

    dfinal = df.merge(df_body, on="link", how = 'inner')
    company = company.replace("+"," ").replace("%26","&").replace("%27","'")
    dfinal.to_excel(writer, re.sub('[^A-Za-z0-9 _]*', '', company), index=False)
    driver.close()
    ct = datetime.datetime.now()
    print(company+"current time:-", ct)

writer.save()

MOTHERSON SUMI SYSTEMS current time:- 2021-06-06 00:19:01.781307
No results found for CERA+SANITARYWARE+
CERA SANITARYWARE current time:- 2021-06-06 00:19:10.341393
No results found for MACROMILL
MACROMILLcurrent time:- 2021-06-06 00:19:17.936523
No results found for AMARA+RAJA+BATTERIES+
AMARA RAJA BATTERIES current time:- 2021-06-06 00:19:26.896086
No results found for TECAN+GROUP+AG
TECAN GROUP AGcurrent time:- 2021-06-06 00:19:36.857127
AERIE PHARMACEUTICALScurrent time:- 2021-06-06 00:19:47.564520
No results found for CTS+EVENTIM+AG+%26+CO+KGAA
CTS EVENTIM AG & CO KGAAcurrent time:- 2021-06-06 00:19:56.025279
CLICKED :1
CLICKED :2
LPL FINANCIAL HOLDINGScurrent time:- 2021-06-06 00:20:54.423188
CLICKED :1
CLICKED :2
CLICKED :3
CLICKED :4
CLICKED :5
CLICKED :6
CLICKED :7
CLICKED :8
CLICKED :9
CLICKED :10
CLICKED :11
CLICKED :12
CLICKED :13
CLICKED :14
CLICKED :15
CLICKED :16
CLICKED :17
CLICKED :18
CLICKED :19
CHARTER COMMUNICATIONScurrent time:- 2021-06-06 00:28:07.199457
CLICKED :

In [47]:
path = "../../../Working/Extracted/Scraped_Data_4.xlsx"
writer = pd.ExcelWriter(path, engine='openpyxl')

for company in name_url[121:160]:
    driver = webdriver.Chrome(chromedriver)
    url = "https://www.reuters.com/search/news?blob="+company+"&sortBy=date&dateRange=pastYear"    
    driver.get(url)

    for count in range(100):
        value = check_exists_by_xpath(loadNoMoreXpath)
        if value == True:
            break
        else:
            try:
                loadMoreButton = driver.find_element_by_xpath(loadMoreXpath)
            except:
                print("No results found for " + company)
                break
            loadMoreButton.click()
            time.sleep(8)
            count=count+1
            print("CLICKED :"+str(count))
    
    titles=[]
    links=[]
    excerpts=[]
    timestamps=[]

    news_titles = driver.find_elements_by_xpath(".//div[@class='search-result-content']/h3[@class='search-result-title']/a[@href]")
    news_links = driver.find_elements_by_xpath(".//div[@class='search-result-content']/h3[@class='search-result-title']/a[@href]")
    news_excerpts = driver.find_elements_by_xpath((".//div[@class='search-result-content']/div[@class='search-result-excerpt']"))
    news_timestamps = driver.find_elements_by_xpath((".//div[@class='search-result-content']/h5[@class='search-result-timestamp']"))
    
    for title in news_titles:
        titles.append(title.text)
        
    for link in news_links:
        links.append(link.get_attribute('href'))
        
    for excerpt in news_excerpts:
        excerpts.append(excerpt.text)
        
    for timestamp in news_timestamps:
        timestamps.append(timestamp.text)

    df = pd.DataFrame({
        "title": titles,
        "link": links,
        "excerpt": excerpts,
        "timestamp":timestamps
    })    
    
    clean_article_body = []
    for i in links:
        response=requests.get(i)
        #print(response.content)
        contents=response.content
        alltext = BeautifulSoup(contents, "lxml")
        #print(alltext)
        
        article_body=[]
        results = alltext.find_all('p', attrs={'class':'Paragraph-paragraph-2Bgue ArticleBody-para-TD_9x'})
        for d in results:
        #if title is not None:
            article_body.append(d.text.strip())
            cleantext = BeautifulSoup(str(article_body), "lxml").text
            clean_txt = re.sub(r'[?|$|.|!]',r'',cleantext)
        clean_article_body.append(clean_txt)
    
    df_body = pd.DataFrame({
        "link": links,
        "body": clean_article_body
    })    

    dfinal = df.merge(df_body, on="link", how = 'inner')
    company = company.replace("+"," ").replace("%26","&").replace("%27","'")
    dfinal.to_excel(writer, re.sub('[^A-Za-z0-9 _]*', '', company), index=False)
    driver.close()
    ct = datetime.datetime.now()
    print(company+"current time:-", ct)

writer.save()

No results found for ALLAKOS
ALLAKOScurrent time:- 2021-06-06 02:34:44.494778
CLICKED :1
CLICKED :2
CLICKED :3
CLICKED :4
CLICKED :5
CLICKED :6
CLICKED :7
CLICKED :8
CLICKED :9
CLICKED :10
CLICKED :11
CHINA STATE CONSTRUCTION ENGINcurrent time:- 2021-06-06 02:38:47.157635
No results found for SAWAI+PHARMACEUTICAL
SAWAI PHARMACEUTICALcurrent time:- 2021-06-06 02:38:58.446610
HAVELLS INDIAcurrent time:- 2021-06-06 02:39:11.742869
CLICKED :1
CLICKED :2
CLICKED :3
CLICKED :4
CLICKED :5
CLICKED :6
CLICKED :7
CLICKED :8
CLICKED :9
CLICKED :10
CLICKED :11
CLICKED :12
CLICKED :13
CLICKED :14
CLICKED :15
CLICKED :16
CLICKED :17
CLICKED :18
CLICKED :19
CLICKED :20
CLICKED :21
CLICKED :22
CLICKED :23
CLICKED :24
CLICKED :25
CLICKED :26
CLICKED :27
CLICKED :28
CLICKED :29
CLICKED :30
CLICKED :31
CLICKED :32
CLICKED :33
CLICKED :34
CHINA GAS HOLDINGS current time:- 2021-06-06 02:50:32.872198
PERNOD RICARD SAcurrent time:- 2021-06-06 02:50:45.129318
CLICKED :1
CLICKED :2
CLICKED :3
CLICKED :4
CLICKE

In [48]:
path = "../../../Working/Extracted/Scraped_Data_5.xlsx"
writer = pd.ExcelWriter(path, engine='openpyxl')

for company in name_url[161:200]:
    driver = webdriver.Chrome(chromedriver)
    url = "https://www.reuters.com/search/news?blob="+company+"&sortBy=date&dateRange=pastYear"    
    driver.get(url)

    for count in range(100):
        value = check_exists_by_xpath(loadNoMoreXpath)
        if value == True:
            break
        else:
            try:
                loadMoreButton = driver.find_element_by_xpath(loadMoreXpath)
            except:
                print("No results found for " + company)
                break
            loadMoreButton.click()
            time.sleep(8)
            count=count+1
            print("CLICKED :"+str(count))
    
    titles=[]
    links=[]
    excerpts=[]
    timestamps=[]

    news_titles = driver.find_elements_by_xpath(".//div[@class='search-result-content']/h3[@class='search-result-title']/a[@href]")
    news_links = driver.find_elements_by_xpath(".//div[@class='search-result-content']/h3[@class='search-result-title']/a[@href]")
    news_excerpts = driver.find_elements_by_xpath((".//div[@class='search-result-content']/div[@class='search-result-excerpt']"))
    news_timestamps = driver.find_elements_by_xpath((".//div[@class='search-result-content']/h5[@class='search-result-timestamp']"))
    
    for title in news_titles:
        titles.append(title.text)
        
    for link in news_links:
        links.append(link.get_attribute('href'))
        
    for excerpt in news_excerpts:
        excerpts.append(excerpt.text)
        
    for timestamp in news_timestamps:
        timestamps.append(timestamp.text)

    df = pd.DataFrame({
        "title": titles,
        "link": links,
        "excerpt": excerpts,
        "timestamp":timestamps
    })    
    
    clean_article_body = []
    for i in links:
        response=requests.get(i)
        #print(response.content)
        contents=response.content
        alltext = BeautifulSoup(contents, "lxml")
        #print(alltext)
        
        article_body=[]
        results = alltext.find_all('p', attrs={'class':'Paragraph-paragraph-2Bgue ArticleBody-para-TD_9x'})
        for d in results:
        #if title is not None:
            article_body.append(d.text.strip())
            cleantext = BeautifulSoup(str(article_body), "lxml").text
            clean_txt = re.sub(r'[?|$|.|!]',r'',cleantext)
        clean_article_body.append(clean_txt)
    
    df_body = pd.DataFrame({
        "link": links,
        "body": clean_article_body
    })    

    dfinal = df.merge(df_body, on="link", how = 'inner')
    company = company.replace("+"," ").replace("%26","&").replace("%27","'")
    dfinal.to_excel(writer, re.sub('[^A-Za-z0-9 _]*', '', company), index=False)
    driver.close()
    ct = datetime.datetime.now()
    print(company+"current time:-", ct)

writer.save()

THERAVANCE BIOPHARMAcurrent time:- 2021-06-06 04:15:35.899673
CLICKED :1
CLICKED :2
SHAKE SHACKcurrent time:- 2021-06-06 04:16:27.123532
GREE ELECTRIC APPLIANCEScurrent time:- 2021-06-06 04:16:43.661229
CLICKED :1
CLICKED :2
CLICKED :3
CLICKED :4
CLICKED :5
CLICKED :6
CLICKED :7
CLICKED :8
CLICKED :9
CLICKED :10
CLICKED :11
CLICKED :12
CLICKED :13
CLICKED :14
CLICKED :15
CLICKED :16
CLICKED :17
CLICKED :18
CLICKED :19
CLICKED :20
CLICKED :21
CLICKED :22
CLICKED :23
CLICKED :24
CLICKED :25
CLICKED :26
CLICKED :27
CLICKED :28
CLICKED :29
CLICKED :30
CLICKED :31
CLICKED :32
CLICKED :33
CLICKED :34
CLICKED :35
CLICKED :36
CLICKED :37
CLICKED :38
CLICKED :39
CLICKED :40
CLICKED :41
CLICKED :42
CLICKED :43
CLICKED :44
CLICKED :45
CLICKED :46
CLICKED :47
CLICKED :48
CLICKED :49
CLICKED :50
CLICKED :51
CLICKED :52
CLICKED :53
CLICKED :54
CLICKED :55
CLICKED :56
CLICKED :57
CLICKED :58
CLICKED :59
CLICKED :60
CLICKED :61
CLICKED :62
CLICKED :63
CLICKED :64
CLICKED :65
CLICKED :66
CLICKED :67
CL

### Save Data to Excel

In [85]:
from openpyxl import load_workbook

li = []

for file_counter in range(1,6):
    file_name = '../../../Working/Extracted/Scraped_Data_' + str(file_counter) + '.xlsx'
    wb = load_workbook(file_name, read_only=True)
    print(file_name)
    for sheet_name in wb.sheetnames:
        try:
            df = pd.read_excel(file_name, sheet_name, index_col=None)
            df.insert(len(df.columns), 'company', sheet_name.strip())
            li.append(df)
        except:
            print('Error occured in -> Filename : "{0}", Sheetname : "{1}"'.format(file_name, sheet_name))
            continue

frame = pd.concat(li, axis=0, ignore_index=True)

Extracted/Scraped_Data_1.xlsx
Extracted/Scraped_Data_2.xlsx
Extracted/Scraped_Data_3.xlsx
Extracted/Scraped_Data_4.xlsx
Extracted/Scraped_Data_5.xlsx


In [86]:
frame.to_excel('../../DataSet/Final.xlsx', index=False)